# Environment Setup  

Specify tensorflow version. The method in this notebook only support TF 1.14 and above. Otherwise other installation method is required.

In [0]:
!pip3 install tensorflow==1.14 ## Change Here
!pip install -U PyYAML

## Configure Tensorflow version

In [0]:
# Restart runtime using 'Runtime' -> 'Restart runtime...'
# %tensorflow_version 2.x
%tensorflow_version 1.x
import tensorflow as tf

print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Mount Google Drive

In [0]:
import os
try:
  from google.colab import drive 
  drive.mount("/content/drive")
  IN_COLAB = True
except:
  IN_COLAB = False

## Import Python Libraries  

In [0]:
import numpy as np
import os
import six.moves.urllib as urllibcategory_index
import sys
import tarfile
import tensorflow as tf
import zipfile
from tqdm import tqdm
from shutil import copy2

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

## Setup Path

In [0]:
PROJ_PATH = '/content/drive/My Drive/NYP_Coursework/ITI110/'

LINK_PATH = '/content/ITI110/'
LOCAL_PATH = '/content/tfod/'
TFOD_PATH = '/content/models/research/'
DATA_PATH = os.path.join(LINK_PATH, 'data')
## Export environment variables
os.environ['PROJ_PATH'] = PROJ_PATH
os.environ['DATA_PATH'] = DATA_PATH
os.environ['LINK_PATH'] = LINK_PATH
os.environ['TFOD_PATH'] = TFOD_PATH

## Setup Link Directory

In [0]:
%%shell

## Source Path 
export GDRIVE_PROJ_PATH=$PROJ_PATH
export LINK_PROJ_PATH="/content/"

echo $GDRIVE_PROJ_PAT
ln -s "$GDRIVE_PROJ_PATH" $LINK_PROJ_PATH

# Install TFOD Library from Github  
Download model from zoo. This step is required everytime the session died. Setup and installation would take about 5 minutes.

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
#!pip install jupyter
#!pip install matplotlib

!git clone https://github.com/tensorflow/models.git

%cd /content/models/research

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

!python object_detection/builders/model_builder_test.py

### Copy training data from cloud into Colab instance  

Folder Structure:  
|--/content/  
|----tfod/  
|--------data/  
|------------images/  
|------------annotations/  
|------------label_map.pbtxt  
|------------train.record-00000-of-xxxxx  
|------------validation.record-00000-of-xxxxx  
|--------models/  
|------------model configuration.config

In [0]:
%%shell
mkdir  /content/tfod/
mkdir  /content/tfod/data/
mkdir  /content/tfod/models/
cp -rf $LINK_PATH/data/ /content/tfod/
cp -rf $LINK_PATH/models/*.config /content/tfod/models/

## Create TF Record  

* if have new records uploaded then have to re-run this step.



In [0]:
%%shell
cd $LINK_PATH
python3 create_tf_record_colab.py --data_dir $LINK_PATH/data/ \
        --output_dir $LINK_PATH/data/ \
        --label_map_path $LINK_PATH/data/label_map.pbtxt \
        --tfod /content/models/research/ \
        --num_shards 1

### Download Pre-trained Model (COCO on SSD Mobile)  
Download model from zoo. This step is required everytime after session died or colab instance re-initiated. 

Folder Structure:  
|--/content/  
|----pretrained_model/  
|--------Model Zoo files



In [0]:
%%shell
## Download the model
rm -r /content/pretrained_model/
mkdir /content/pretrained_model/
cd /content/pretrained_model/
wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz

## unzip the model.tar.gz file 
tar -xvf ssd_mobilenet_v2_coco_2018_03_29.tar.gz

## Run TFOD Training  
* When using same model output folder, TFOD will resume progress from previous training  
* Checkpoint will directly output to Google Drive if the session died. THe drawback is this will take up Google Drive space. However, TFOD have rolling checkpoint functionality which mean old checkpoints will move to bin. Once the Drive is full, old checkpoints will be automatically remove from bin (Unrecoverable).

In [0]:
%%shell
# Example from the /content/models/research/ directory
PIPELINE_CONFIG_PATH=/content/tfod/models/ssd_mobilenet_v2_coco.config

## Directly save checkpoint at gdrive
MODEL_DIR=/content/ITI110/models/ssd_coco_3_20_20/
NUM_TRAIN_STEPS=2000
SAMPLE_1_OF_N_EVAL_EXAMPLES=10

python3 /content/models/research/object_detection/model_main.py \
    --pipeline_config_path=${PIPELINE_CONFIG_PATH} \
    --model_dir=${MODEL_DIR} \
    --num_train_steps=${NUM_TRAIN_STEPS} \
    --sample_1_of_n_eval_examples=$SAMPLE_1_OF_N_EVAL_EXAMPLES \
    --alsologtostderr

# Export 
By default, TFOD training will only keep a few checkpoints. It would be better to run this portion at seperate notebook to export fronzen graph and let training continue to run.

## TF Frozen Graph

In [0]:
%%shell

PIPELINE_CONFIG_PATH=/content/ITI110/models/ssd_mobilenet_v2_coco.config
MODEL_DIR=/content/ITI110/models/model_ssd_coco/

python3 /content/models/research/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path  ${PIPELINE_CONFIG_PATH} \
    --trained_checkpoint_prefix /content/ITI110/models/model_ssd_coco/model.ckpt-1373  \
    --output_directory  /content/ITI110/models/exported_graphs 

## SSD Graph

In [0]:
%%shell 

cd  /content/models/research/object_detection

python3 export_tflite_ssd_graph.py \
--pipeline_config_path=/content/ITI110/models/ssd_mobilenet_v2_coco.config \
--trained_checkpoint_prefix=/content/ITI110/best/model.ckpt-3097 \
--output_directory=/content/ITI110/tflite/

## TFLite Conversion

In [0]:
%%shell 

cd  /content/models/research/object_detection

tflite_convert \
--graph_def_file=/content/ITI110/tflite/tflite_graph.pb \
--output_file=/content/ITI110/tflite/detect.tflite \
--output_format=TFLITE \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--mean_values=128 \
--std_dev_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops

# Tensorboard  

* It would be better to run this part at a seperate notebook for progress monitoring
* It would better to put log directory at google drive for parallel access

In [0]:
%load_ext tensorboard
%tensorboard --logdir /content/ITI110/models/ssd_coco_3_20_20/